In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1768319851381, experiment_id='2', last_update_time=1768319851381, lifecycle_stage='active', name='my-nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1768238871472, experiment_id='0', last_update_time=1768238871472, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = "2",
    filter_string = "",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 21bd2413b0c5495b9ce390d0e09802e7, rmse: 6.3286
run id: 578544cf4ecd4c64b77e3c2091ff438d, rmse: 6.3690
run id: 9626268a89af405fb4fd7525d8c67107, rmse: 6.4159
run id: d3766ec74fa541e9bdde398a55a6b5bf, rmse: 6.4159
run id: 71c44b2e1c904fcda790a323f3c4b9c9, rmse: 6.4159


In [10]:
import mlflow

In [11]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
moldel_uri = "mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts"

mlflow.register_model(model_uri=moldel_uri, name="nyc-taxi regressor")

Registered model 'nyc-taxi regressor' already exists. Creating a new version of this model...
2026/01/14 10:31:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi regressor, version 3
Created version '3' of model 'nyc-taxi regressor'.


<ModelVersion: aliases=[], creation_timestamp=1768386693384, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1768386693384, metrics=None, model_id=None, name='nyc-taxi regressor', params=None, run_id='', run_link='', source='mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts', status='READY', status_message=None, tags={}, user_id='', version='3'>

In [15]:
model_name="nyc-taxi regressor"

client.get_latest_versions(name=model_name)

/tmp/ipykernel_34185/1069848613.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1768386693384, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1768386693384, metrics=None, model_id=None, name='nyc-taxi regressor', params=None, run_id='', run_link='', source='mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts', status='READY', status_message=None, tags={}, user_id='', version='3'>]

In [17]:
model_version=3
model_alias="production"

client.set_registered_model_alias(
    name=model_name,
    alias=model_alias, 
    version=model_version
)

In [18]:
from datetime import datetime

In [20]:
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {model_alias} on {date}"
)

<ModelVersion: aliases=['production'], creation_timestamp=1768386693384, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 3 was transitioned to production on 2026-01-14', last_updated_timestamp=1768387488657, metrics=None, model_id=None, name='nyc-taxi regressor', params=None, run_id='', run_link='', source='mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts', status='READY', status_message=None, tags={}, user_id='', version='3'>

In [30]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(model_uri, X_test, y_test):
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred)**0.5}

In [22]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')

In [23]:
run_id = "9626268a89af405fb4fd7525d8c67107"

In [24]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/MLOps/02-experiment-tracking/preprocessor'

In [25]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [26]:
X_test = preprocess(df, dv)

In [27]:
target = "duration"
y_test = df[target].values

In [31]:
model_uri = "mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts"

In [32]:
%time test_model(model_uri, X_test=X_test, y_test=y_test)

CPU times: user 3.11 s, sys: 92.7 ms, total: 3.2 s
Wall time: 2.12 s


{'rmse': 6.353322649178022}